In [2]:
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds

C:\Users\aleks\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ModuleNotFoundError: No module named 'resource'

In [3]:
df_loc1 = pd.read_csv("C:/Users/aleks/OneDrive/Documents/flutter_projects/power_generation_data/location1_final.csv")
df_loc2 = pd.read_csv("C:/Users/aleks/OneDrive/Documents/flutter_projects/power_generation_data/location2_final.csv")
df_loc3 = pd.read_csv("C:/Users/aleks/OneDrive/Documents/flutter_projects/power_generation_data/location3_final.csv")
df_loc4 = pd.read_csv("C:/Users/aleks/OneDrive/Documents/flutter_projects/power_generation_data/location4_final.csv")

In [4]:
power_df = pd.concat([df_loc1, df_loc2, df_loc3, df_loc4])

In [5]:
power_df.head()

,temperature_2m,windspeed_100m,sin_wd_100,Power
0,28.5,1.26,0.309017,0.1635
1,28.4,3.99,0.374607,0.1424
2,26.8,2.78,0.500000,0.1214
3,27.4,2.69,0.965926,0.1003
4,27.3,4.43,0.994522,0.0793


In [6]:
rows, columns = power_df.shape
print(f'Number of rows: {rows}')
print(f'Number of columns: {columns}')

Number of rows: 175200
Number of columns: 4


We have 175200 training examples from the concatenated dataframe.

In [7]:
X = power_df.drop('Power', axis=1).values
y = power_df['Power'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape = (X_train.shape[1],)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

Training the model:

In [9]:
history = model.fit(X_train, y_train, epochs = 10, validation_split = 0.2)

Epoch 1/10

3504/3504 [==============================] - 18s 4ms/step - loss: 0.1488 - val_loss: 0.0407
Epoch 2/10
3504/3504 [==============================] - 14s 4ms/step - loss: 0.0419 - val_loss: 0.0405
Epoch 3/10
3504/3504 [==============================] - 11s 3ms/step - loss: 0.0420 - val_loss: 0.0406
Epoch 4/10
3504/3504 [==============================] - 10s 3ms/step - loss: 0.0420 - val_loss: 0.0405
Epoch 5/10
3504/3504 [==============================] - 10s 3ms/step - loss: 0.0415 - val_loss: 0.0409
Epoch 6/10
3504/3504 [==============================] - 9s 2ms/step - loss: 0.0410 - val_loss: 0.0403
Epoch 7/10
3504/3504 [==============================] - 9s 3ms/step - loss: 0.0408 - val_loss: 0.0394
Epoch 8/10
3504/3504 [==============================] - 10s 3ms/step - loss: 0.0406 - val_loss: 0.0415
Epoch 9/10
3504/3504 [==============================] - 9s 3ms/step - loss: 0.0405 - val_loss: 0.0418
Epoch 10/10
3504/3504 [==============================] - 9s 3ms/step - loss

Evaluating the model:

In [10]:
test_loss = model.evaluate(X_test, y_test)
predictions = model.predict(X_test)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)

1095/1095 [==============================] - 2s 2ms/step


In [14]:
print(f"MAE: {mae}")
print(f"MSE: {mse}")

MAE: 0.14919286269063303
MSE: 0.040098718157223406


In [15]:
model.save('saved_model/power_generation_predictor')

INFO:tensorflow:Assets written to: saved_model/power_generation_predictor\assets


INFO:tensorflow:Assets written to: saved_model/power_generation_predictor\assets


In [16]:
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model/power_generation_predictor')
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)